<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/5-autokeras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/5-autokeras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# [AutoKeras](https://autokeras.com/)

** Note: This code has been tested as of June 2020**

As AI is automating more and more jobs, it can finally automate designing AI architectures too. Neural Architecture Search (NAS) approaches utilize reinforcement learning to join together mini architectural blocks, till they are able to maximize the objective function - i.e. our validation accuracy. The current state of the art networks are all based on NAS, leaving human-designed architectures in the dust. Research in this area started showing promising results in 2017, with a bigger focus on making train faster in 2018.

AutoKeras (Haifeng Jin et al), also apply this state of the art technique on our particular datasets in a relatively accessible manner. Generating new model architectures with AutoKeras is a matter of supplying our images and associated labels, and a time limit to finish running the jobs by. Internally, it implements several optimization algorithms including a Bayesian optimization approach to search for an optimal architecture.

We start off by loading the necessary packages. If you are running this script on your local machine then please install `autokeras` and `kerastuner` and then start Jupyter using the following commands: 

```
pip install autokeras
pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc0
```

If  you are running on Google Colab, they will be installed automatically.

In [ ]:
try:
    import google.colab
    IS_COLAB_ENV = True

In [ ]:
if IS_COLAB_ENV:
    !pip install autokeras
    !pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc0

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak

Load the standard MNIST data

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Define an image classifier. The `max_trials` setting can be played around with to enable more experimentation. It refers to the maximum number of different Keras Models to try. The search may finish before reaching the max_trials. Defaults to 100.

In [ ]:
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=10)

Fit the classifier.

In [ ]:
clf.fit(x_train, y_train, epochs=10)

Prediction based on the best model trained

In [ ]:
predicted_y = clf.predict(x_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_true=y_test, y_pred=predicted_y))

Evaluate the best model with testing data

In [ ]:
print(clf.evaluate(x_test, y_test))

Export as a Keras Model

In [ ]:
model = clf.export_model()

Let's visualize the trained model.

In [ ]:
model.summary()

In [ ]:
print(type(model))

Now, let's write the model to disk

In [ ]:
try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")

Let's read the model we just wrote.

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))